In [1]:
from pyiron_base import Project
from pyiron_base.project.delayed import DelayedObject
from python_workflow_definition.pyiron_base import load_workflow_json, _get_unique_objects, _get_connection_dict, _get_edges_dict
from python_workflow_definition.shared import (
    NODES_LABEL,
    EDGES_LABEL,
    VERSION_NUMBER,
    VERSION_LABEL,
    set_result_node,
    update_node_names,
)
import json
import numpy as np
from inspect import isfunction

# Clean up 

In [2]:
pr = Project(".")

In [3]:
pr.remove_jobs(recursive=True, silently=True)

0it [00:00, ?it/s]

# Load Workflow

In [4]:
workflow_json_filename = "workflow.json"

In [6]:
delayed_object_lst = load_workflow_json(file_name=workflow_json_filename)
# delayed_object_lst[-1].draw()

# Run Workflow

In [7]:
new_lst = []
for d in delayed_object_lst:
    d.execute_in_working_directory = True
    new_lst.append(d)

In [ ]:
new_lst[-1].pull()

The job get_bulk_structure_2a7f41a00815f6bf9ba52bd8837bace5 was saved and received the ID: 1
The job get_dict_5aee89acb3a4017b05c39f7477a082f8 was saved and received the ID: 2
The job calculate_qe_8649213786bf31b0c67e1cc4640b5cee was saved and received the ID: 3


[jupyter-jan-janssen-pwd-qe-data-export-kum87rd0:00420] mca_base_component_repository_open: unable to open mca_btl_openib: librdmacm.so.1: cannot open shared object file: No such file or directory (ignored)


# Write Workflow with Output

In [ ]:
def _remove_server_obj(nodes_dict: dict, edges_lst: list):
    server_lst = [k for k in nodes_dict.keys() if k.startswith("_server_obj_")]
    for s in server_lst:
        del nodes_dict[s]
        edges_lst = [ep for ep in edges_lst if s not in ep]
    return nodes_dict, edges_lst

In [ ]:
def _get_nodes(connection_dict: dict, delayed_object_updated_dict: dict):
    return {
        connection_dict[k]: v._python_function if isinstance(v, DelayedObject) else v
        for k, v in delayed_object_updated_dict.items()
    }

In [ ]:
def _get_working_directories(connection_dict: dict, delayed_object_updated_dict: dict):
    return {
        connection_dict[k]: v._job.working_directory 
        for k, v in delayed_object_updated_dict.items() if isinstance(v, DelayedObject)
    }

In [ ]:
def _get_output(connection_dict: dict, delayed_object_updated_dict: dict):
    return {
        connection_dict[k]: v._job.output.to_builtin()["result"] 
        for k, v in delayed_object_updated_dict.items() if isinstance(v, DelayedObject)
    }

In [ ]:
def get_input_obj(obj):
    obj_lst = []
    for obj_in in obj.input.values():
        if isinstance(obj_in, DelayedObject):
            obj_lst += [obj_in]
            obj_lst += get_input_obj(obj=obj_in)
    return obj_lst

In [ ]:
def write_workflow_json(
    delayed_object: DelayedObject, file_name: str = "workflow.json"
):
    nodes_dict, edges_lst = delayed_object.get_graph()
    nodes_dict, edges_lst = _remove_server_obj(
        nodes_dict=nodes_dict, edges_lst=edges_lst
    )
    delayed_object_updated_dict, match_dict = _get_unique_objects(nodes_dict=nodes_dict)
    connection_dict, lookup_dict = _get_connection_dict(
        delayed_object_updated_dict=delayed_object_updated_dict, match_dict=match_dict
    )
    nodes_new_dict = _get_nodes(
        connection_dict=connection_dict,
        delayed_object_updated_dict=delayed_object_updated_dict,
    )
    output_dict = _get_output(
        connection_dict=connection_dict,
        delayed_object_updated_dict=delayed_object_updated_dict,
    )
    working_dict = _get_working_directories(
        connection_dict=connection_dict,
        delayed_object_updated_dict=delayed_object_updated_dict,
    )
    edges_new_lst = _get_edges_dict(
        edges_lst=edges_lst,
        nodes_dict=nodes_dict,
        connection_dict=connection_dict,
        lookup_dict=lookup_dict,
    )

    nodes_store_lst = []
    for k, v in nodes_new_dict.items():
        if isfunction(v):
            mod = v.__module__
            if mod == "python_workflow_definition.pyiron_base":
                mod = "python_workflow_definition.shared"
            nodes_store_lst.append(
                {"id": k, "type": "function", "value": mod + "." + v.__name__, "output": output_dict[k], "working_directory": working_dict[k]}
            )
        elif isinstance(v, np.ndarray):
            nodes_store_lst.append({"id": k, "type": "input", "value": v.tolist()})
        else:
            nodes_store_lst.append({"id": k, "type": "input", "value": v})

    content = set_result_node(
        workflow_dict=update_node_names(
            workflow_dict={
                VERSION_LABEL: VERSION_NUMBER,
                NODES_LABEL: nodes_store_lst,
                EDGES_LABEL: edges_new_lst,
            }
        )
    )
    
    with open(file_name, 'w') as f:
        json.dump(content, f, indent=4)

In [ ]:
write_workflow_json(
    delayed_object=new_lst[-1], file_name="workflow_with_output.json"
)

In [ ]:
!cat workflow_with_output.json